In [28]:
import csv
from openai import OpenAI
import os
import re

# OPENAI_API_KEYを含むとPushできないため実行時は有効にしてください
# os.environ["OPENAI_API_KEY"] = "*****"

# openAIの機能をclientに代入
client = OpenAI()

# 対象者のMBTIタイプ
user_mbti = "INTJ"

In [29]:
# CSVファイルを読み込み
with open('output.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    people = list(reader)

In [30]:
def find_best_matches(people, user_mbti, top_n=3):
    prompts = []
    persons = ""
    for person in people:
        persons = persons + str(f"名前: {person['名前']} 得意: {person['得意']}, 不得意: {person['不得意']},")
    prompt = f"MBTIタイプが{user_mbti}の人物と合う人物を以下から3名選んでください。 " + persons + "。対象者は、名前：,選定理由:形式で出力してください"
    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt },
        ],
    )
        
    # コンテンツを抽出
    content = response.choices[0].message.content.strip()
    print(content)

    # 対象者と選定理由を抽出するための正規表現パターン
    pattern = r"対象者は、名前：([^,]+), 選定理由: ([^\n]+)"
    # 正規表現による抽出
    matches = re.findall(pattern, content)
    # 結果を配列に格納
    result = [{"名前": match[0], "選定理由": match[1]} for match in matches]

    return result

In [31]:
# 最適な3人を選択
best_matches = find_best_matches(people, user_mbti)

# 結果を表示
for match in best_matches:
     print(match)

対象者は、以下の3名です。

1. 名前: 田中太郎  
   選定理由: 論理的思考が得意で、INTJの人物と共通の思考スタイルを持っています。感情表現が不得意なため、INTJの合理的なアプローチに理解を示す可能性があります。

2. 名前: 小林洋介  
   選定理由: 戦略的計画を得意とし、INTJの特性である長期的な視点や計画性にマッチします。感情的な対話が不得意なので、理論的な議論が中心のコミュニケーションがしやすいです。

3. 名前: 伊藤和也  
   選定理由: 技術的な問題解決が得意で、INTJの分析的な特性と相性が良いです。対人スキルが不得意なため、データや論理に基づく議論に特化しやすい環境を提供できます。
